# 문자를 읽을 수 있는 딥러닝


## 기계가 읽을 수 있나요?

![Img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-1.max-800x600_gPtqrYi.jpg)

사람이 문자를 읽으려면 어떤 과정을 거칠까요? 문자가 있다는 것을 1)발견하고, 발견한 문자를 2)해독하는 과정을 거치는 것이 일반적일 텐데요,
기계가 문자를 읽는 과정도 이와 다르지 않을 것입니다. 컴퓨터 비전에서 사용하는 용어로 위의 두 단계를 번역하자면 문자의 존재를 1)Detection 어떤 문자인지 판독하는 2)Recognition 으로 나눌 수 있을 것입니다.


## 어떤 과정으로 읽을까요?

구글 API에서는 문자의 영역을 사각형으로 표현하고 우측에 "Block"과 "Paragraph"로 구분해서 인식결과를 나타내고 있었습니다. 구글 API가 이미지에 박스를 친 다음 박스별 텍스트의 내용을 알려준 것처럼, 문자 모델은 보통 두 단계로 이뤄집니다. 먼저 입력받은 사진 속에서 문자의 위치를 찾아냅니다. 이 과정을 **문자 검출(text detection)** 이라고 합니다. 이렇게 찾은 문자 영역으로부터 문자를 읽어냅니다. 이 과정을 **문자 인식(text recognition)** 이라고 합니다. 예를 들어 아래 카카오의 OCR 모델은 먼저 문자가 있는 영역의 정보 (`coord`)를 찾아내고, 각 영역에서 문자를 인식하고 있습니다. 이렇게 문자 인식 모델은 문자 검출과 문자 인식 두 과정을 통해서 사진 속의 문자를 읽을 수 있게 됩니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-4.max-800x600_lhFXcl2.png)

위 그림에서는 문자의 영역을 표현하는 방법으로 사각형의 네 꼭지점 좌표를 알려 주는 방법을 제시하고 있습니다. 하지만 다른 도형이나 표현 방법을 사용할 수도 있을 것입니다. 


## 딥러닝 문자 인식의 시작

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-5.max-800x600_fSsqPX1.png)

다들 딥러닝을 배우는 동안 MNIST 데이터셋을 여러 번 접하셨을 것입니다. 주로 0부터 9까지의 손으로 쓰인 숫자가 있는 이미지를 이미지 분류 문제로 딥러닝을 이용하여 풀어보는 데이터셋으로 사용하지요!

MNIST가 딥러닝 데이터셋의 "hello, world!"라면, LeNet은 CNN의 "hello, world!"라 할 수 있습니다. 1998년에 발표된 LeNet은 우편번호나 손글씨를 읽기 위해서 만들어졌습니다. 앞서 학습한 MNIST로 LeNet을 훈련시킨다면 우편번호를 읽을 수 있는 것이지요. 위 그림은 LeNet-5의 구조입니다. 요즘 나온 모델들을 생각한다면 구조가 상당히 간단하지요? Convolution 레이어와 최종 출력 레이어로 이뤄져 있습니다.

아래 이미지는 입력과 각 Layer의 활성화를 시각화한 LeNet의 MNIST 데모 이미지입니다. 우측 상단에서 인식결과를 확인할 수 있습니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/original_images/E-19-6.gif)

아래 데모사이트에서 이렇게 간단한 구조로도 어려운 글자를 읽을 수 있는 딥러닝 분류 모델, LeNet을 확인해 보세요.


- [Yann LeCun's Demo](http://yann.lecun.com/exdb/lenet/stroke-width.html)
- [L12/1 LeNet from Berkely](https://www.youtube.com/watch?v=m3BrTjo2zUA&feature=youtu.be)

하지만 이렇게 단순한 분류 모델만으로는 우리가 위에서 구글 API로 테스트해 보았던 복잡한 결과를 얻을 수 없습니다. 넓고 복잡한 이미지에서 글자 영역을 찾을 수 없을 뿐더러 글자를 영역별로 잘라서 넣더라도 우리가 인식하기를 원하는 사진은 여러 글자가 모여있기 때문에 단순한 분류 문제로 표현이 불가능하지요.

## 사진 속 문자 찾아내기 - detection

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-10.max-800x600_iGdm4cV.png)

사진 속 문자를 찾아내는 딥러닝 모델은 최근에는 일반적인 객체 검출(object detection) 방법으로 접근합니다. 이미지 속에서 물체를 찾아내는 딥러닝 모델을 문자를 찾도록 학습을 시킨 것이지요. 물론 문자의 특성에 따라서 모델을 변경해주기도 한답니다.

딥러닝 기반 객체 검출 방법에는 회귀(regression) 방식이나 세그멘테이션(segmentation) 방식이 있습니다. **회귀 방식**은 기준으로 하는 박스에 대비 문자의 박스가 얼마나 차이가 나는지를 학습합니다. **세그멘테이션 방식**은 픽셀단위로 해당 픽셀이 문자를 표현하는지를 분류하는 문제(pixel-wise classification)로 푼다고 볼 수 있습니다. 객체 검출과 세그멘테이션에 대한 자세한 방법은 다른 강의에서 배우도록 합시다. 여기서는 문자를 찾아내는 딥러닝 모델이 우리가 흔히 들어본 객체 검출(object detection)과 큰 차이가 없다는 것만 알아두셔도 됩니다.

- 참고 : [딥러닝을 활용한 객체 탐지 알고리즘 이해하기](https://blogs.sas.com/content/saskorea/2018/12/21/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%EA%B0%9D%EC%B2%B4-%ED%83%90%EC%A7%80-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0/)

|사물 인식|글자 인식|
|:---:|:---:|
|![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-9.max-800x600_lt28Nt5.jpg)|![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-8.max-800x600_wGIgZfi.jpg)|


## 사진 속 문자 읽어내기 - recognition

![img](https://aiffelstaticprd.blob.core.windows.net/media/original_images/E-19-11.png)

문자 인식은 사진 속에서 문자를 검출해내는 검출 모델이 영역을 잘라서 주면 그 영역에 어떤 글자가 포함되어있는지 읽어내는 과정입니다. 아래는 ICDAR15라는 OCR 데이터셋에서 단어 단위로 잘린 이미지입니다. 문자 인식 모델은 이렇게 작게 잘린 이미지가 들어오면 이미지 속에 어떤 단어가 포함됐는지 찾아내지요.

사실 이 과정은 이미지 문제보다는 자연어처리에서 많은 영감을 받았습니다. 자연어 처리를 한다면 RNN이 떠오르실 텐데요. 이미지 내의 문자 인식 모델의 기본적인 방법 중 하나가 바로 CNN과 RNN을 결합한 CRNN 모델입니다. 이미지 내의 텍스트와 연관된 특징을 CNN을 통해 추출한 후에 스텝 단위의 문자 정보를 RNN으로 인식하는 것이지요.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-19-12.max-800x600_y9DP2Mu.png)

이렇게 검출 결과로부터 인식 모델을 통해 문자를 인식하고 나면 구글 API의 결과처럼 사진 속 영역에 표시를 하고 그 영역에 어떤 문자가 적혀있는지 알 수 있는 OCR 시스템이 됩니다.

이상으로 OCR에서 고려해야 할 기술적 단계들의 개요만 아주 간단히 짚어 보았습니다. OCR은 이미 실용적인 단계로의 연구가 많이 진척되어, 실전 레벨의 구체적인 정보를 접하는 것이 훨씬 도움이 될 것입니다.

그래서 OCR 영역에서 아주 도움이 될만한 영상을 권해 드립니다. 둘 다 한국어 영상이라 부담없이 보실 수 있습니다. 네이버 이활석님의 데뷰 2018 발표영상은 네이버 OCR팀의 값진 연구 성과와 그동안의 생생한 경험을 통해 OCR 분야에 대한 구체적인 안내를 얻을 수 있는 아주 훌륭한 자료입니다. 꼭 시청하시기를 권해 드립니다.

엄태웅님의 영상은 이활석님의 네이버 데뷰 2018 영상을 기반으로 정리한 자료에 본인만의 개념적인 설명을 곁들여 OCR의 접근 방식에 대해 설명해 주십니다.함께 보시면 도움이 될 것입니다.

- [네이버 데뷰 2018, 이활석님의 CRAFT 모델소개와 연구 경험](https://tv.naver.com/v/4578167)
- [Terry TaeWoong Um님의 사진 속 글자 읽기, OCR (Optical character recognition)](https://www.youtube.com/watch?v=ckRFBl_XWFg&ab_channel=TerryTaeWoongUm)